# Notebook 6 · Verified RAG

This notebook attaches a verification stage that checks generated claims against secondary retrieval evidence.

In [ ]:
from typing import Dict

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI as LangChainChatOpenAI

from pprint import pprint

from shared import (
    DEFAULT_MODEL,
    RetrievalContext,
    build_baseline_chain,
    build_retrieval_context,
    pretty_print_json,
    time_execution,
)


In [ ]:
context = build_retrieval_context(top_k=4)
qa_chain = build_baseline_chain(context.retriever)
verifier_llm = LangChainChatOpenAI(model=DEFAULT_MODEL, temperature=0.0)
verification_prompt = PromptTemplate.from_template(
    'Statement: {statement}\nEvidence: {evidence}\n\nIs the statement fully supported by the evidence? Reply with SUPPORTED, PARTIAL, or UNSUPPORTED and explain.'
)


In [ ]:
def answer_with_verification(question: str) -> Dict[str, str]:
    docs = context.retriever.get_relevant_documents(question)
    evidence = '\n\n'.join(doc.page_content for doc in docs)
    answer = qa_chain.run(question)
    verdict = verifier_llm.predict(verification_prompt.format(statement=answer, evidence=evidence))
    return {'answer': answer, 'verdict': verdict}

result = answer_with_verification('Summarise the security features included in the Enterprise plan.')
print(result['answer'] + '\n\nVerification verdict: ' + result['verdict'])


## Next steps

* Implement granular claim extraction before verification.
* Route UNSUPPORTED claims back to the retriever for additional sourcing.
* Log verification outcomes to build observability dashboards.